# Capacitive simulation of qubit decay in charge line
Optimize the T1 limit of the qubit as a function of its distance from a charge line.

**_NOTE:_**  In order for the optimization to work properly, the design variable ```design_var_cl_pos_x_qubit``` refers to the relative position between the charge line and the edge of the transmon's pocket. Check the [design file](https://github.com/202Q-lab/QDesignOptimizer/blob/main/projects/examples_coupled_transmon_chip/design.py) to see how it is parsed when creating the design.

In [ ]:
%load_ext autoreload
%autoreload 2

## Rendering the design

In [ ]:
import names as n
from design import render_qiskit_metal_design
from qdesignoptimizer.utils.chip_generation import create_chip_and_gui, ChipType

chip_type = ChipType(size_x="10mm", size_y="10mm", size_z="-300um", material="silicon")
design, gui = create_chip_and_gui(n.CHIP_NAME, chip_type, open_gui=True)
render_qiskit_metal_design(design, gui)

## Creating the study and optimization targets

In [ ]:
import mini_studies as ms
import optimization_targets as ot

In [ ]:
MINI_STUDY_GROUP = n.NBR_1
MINI_STUDY = ms.get_mini_study_qb_charge_line(group=MINI_STUDY_GROUP)
RENDER_QISKIT_METAL = lambda design: render_qiskit_metal_design(design, gui)

opt_targets = ot.get_opt_targets_qb_charge_line(group=MINI_STUDY_GROUP)

## Creating design analysis objects

In [ ]:
import time
import parameter_targets as pt
import plot_settings as ps

from qdesignoptimizer.design_analysis import DesignAnalysis, DesignAnalysisState
from qdesignoptimizer.utils.utils import get_save_path

# Closing Ansys hfss
from qdesignoptimizer.utils.utils import close_ansys

close_ansys()

In [ ]:
design_analysis_state = DesignAnalysisState(
    design, RENDER_QISKIT_METAL, pt.PARAM_TARGETS
)

design_analysis = DesignAnalysis(
    design_analysis_state,
    mini_study=MINI_STUDY,
    opt_targets=opt_targets,
    save_path=get_save_path("out/", n.CHIP_NAME),
    update_design_variables=False,
    plot_settings=ps.PLOT_SETTINGS_CHARGE_LINE_DECAY,
)

## Optimization step

In [ ]:
# number of runs of optimization and number of passes for simulation at each run
group_runs = 10
group_passes_cap = 8

for i in range(group_runs):
    design_analysis.update_nbr_passes_capacitance_ministudies(group_passes_cap)
    design_analysis.optimize_target({}, {})
    design_analysis.screenshot(gui=gui, run=i)

## Update parameters

In [ ]:
design_analysis.overwrite_parameters()

## Close

In [ ]:
close_ansys()